# Reinforcement learning


## Modell basiert

* Was bedeutet Modell basiert?

## Modell frei
* Was bedeutet Modell frei?

### Passives lernen vs. aktives lernen
Beides Arten von Reinforcement learning

Passiv:
* Agent hat die optimale Policy
* Agent weiß, was er machen muss
* Ziel: feste Abfolge von Aktionen ausführen, wie gut ist die optimale Policy?
* ADP, TD

Aktiv:
* Agent kennt die optimale Policy nicht
* Agent muss entscheiden, was er tun muss
* Ziel: handeln und optimale Policy lernen
* ADP mit exploration function, Q-learning


### (Adaptive dynamic programming (ADP) )


### Monte-Carlo-Methode

Die Monte-Carlo-Methode ist eine Methode zur Ermittlung der zu erwartenden Belohnung beim Reinforcement Learning. Das besondere bei Monte-Carlo ist, dass es sich um einen randomisierten Algorithmus handelt. Als Vorraussetzung werden Erfahrungswerte, also Trainingsdaten bestehend aus einer Folge von Zuständen, Aktionen und Belohnungen benötigt. Die Idee ist, aus diesen Daten den durchschnittlich erreichten Wert nach besuchen eines Zustands $s$ zu ermitteln. Dazu gibt es zwei Ansätze: "Every-Visit" und "First-Visit".

Bei "First-Visit" werden alle Belohnungen, die zwischen dem ersten Besuch des betrachteten Zustands $s$ und dem Endzustand auftreten, aufsummiert. Die so berechnetetn Werte für alle Episoden werden aufaddiert und durch die Anzahl der betrachteten Episoden geteilt.

**TODO: Beispielgraph einfügen**

Bei "Every-Visit" wird ein neuer Summations-Term für jedes Auftreten des Zustands $s$ erstellt. Das bedeutet man summiert zunächst alle Belohnung zwischen dem ersten Auftreten und dem Endzustand, addiert dazu die Summe der Belohnungen zwischen dem zweiten Auftreten und dem Endzustand und so weiter bis zur Addition der Summe der Belohnungen zwischen dem letzten Auftreten und dem Endzustand. Dies wird für alle Episoden wiederholt, aufaddiert und durch die Anzahl der Summations-Terme, also die Anzahl des Auftretens des Zustands $s$, dividiert.

**TODO: Beispielgraph einfügen**

(vgl. Gupta 2020, Sutton 2015)

In der Anwendung von Monte-Carlo wird eine Episode von einem zufälligen Zustand bis zum Endzustand mit zufälligen Aktionen durchgespielt und dabei Zustände, Aktionen und Belohnungen aufgezeichnet. Anschließend werden die Werte der Zustände nach einer der oben erläuterten Methode berechnet. Anschließend wird ein neuer zufälliger Zustand bestimmt und unter Ausnutzung der zuvor berechneten Werte bis zum Ende geführt. Die Werte werden nach dem Durchgang neu berechnet und gemittelt. Dies wird viele Male wiederholt. So ergibt sich die gewünschte Werte-Matrix Q.

Bei diesem Algorithmus ist nicht sichergestellt, dass jeder Zustand nach dem Training besucht wurde. Ein weiterer Nachteil ist, dass das Problem terminieren muss, bevor die Werte berechnet werden können. Es muss sich also zwingend um terminirende Probleme handeln.

### Temporal-difference learning (TD-Learning)

* Agent passt Policy nach jeder Aktion auf Basis einer geschätzten erwarteten Belohnung an (nicht erst, wenn er die Belohnung erhält)
* Funktion $V$ bewertet Zustand $s_t$
* Mit jeder Iteration wird $V$ genauer, d.h. $V$ nähert sich $V^{\pi}$ an
* Lernrate: Wie gut passen neue Werte die Bewertungsfunktion $V$ an?

$V(s_t)\leftarrow(1-\alpha)V(s_t)+\alpha (r_{t+1}  \gamma V(s_{t+1}))$ mit Lernrate $\alpha$ und Discount-Faktor $\gamma$

- mit Gewichtung der neuen Aktionen
- Vorteil: neue Aktionen haben eine höhere Auswirkungen auf das Model, minimiert die Variazn im vergleich zu monte carlo

Ähnlich wie bei Monte Carlo, wird Temporal difference learning zur Berechnung der Q-Matrix verwendet. Bei jedem Durchlauf entscheidet sich der Agent für die Aktion, die zu dem State mit dem höchsten Reward führt. Im Anschluss wird der Wert mit Hilfe der Value-Funktion nach folgender Formel angepasst:
    
$V(s_t)\leftarrow(1-\alpha)V(s_t)+\alpha(r_{t+1} \gamma V(s_{t+1}))$

wobei gilt:
* $V(s_t)$ ist der Wert des aktuellen States
* $r_t+1$ ist der Reward, der im nächstem Schritt erhalten wird
* $\gamma$ ist die Gewichtung, wie hoch der Einfluss des Wertes in den Gesamtwert eingeht
* $\alpha$ ist die Lernrate

Die Matrix optimiert sich direkt nach jedem Trainings-Durchlauf. Im Gegensatz zu Monte Carlo werden die Rewards, mit einem Gewicht $\gamma$ versehen. Dadurch wird aktuelleren Werten eine höhere Wertigkeit im Endergebnis gegeben. Der Vorteil dabei ist, dass die Varianz minimiert wird. Ausreißer haben also kleinere Auswirkungen auf das Endergebnis.

**TODO: Beispielgraph einfügen**

### Q-learning

* Variante von TD-learning

- Modellfreier R.L. Algorythmus um der Wert einer Aktion zu berechnen
- Benötigt keine Anpassung des Enviroments, nut Rewards und stochastische Transitionen
- Für einen Endlichen Markov Decision Prozess, findet Q-learning die Optimale Policy(Regel), um eine optimale Aktionsauswahl zu treffen, auf Basis von zufälligen Aktionen, falls man unendlich Zeit zur Verfügung hat.
- In eine Matrix len(States)xlen(Aktionen) werden die Rewards für eine Aktion im State eingetragen.
- Initial ist die Matrix mit 0 gefüllt
- Die Q-Funktion: Q: S x A -> R mappt eine Aktion und ein State auf eine Rationale zahl, die in der Matrix eingetragen wird
- Neue Werte werden mit einer höheren Gewichtung in der Matirx einberechnet (gamma, discount factor)
- in der Tabelle sind die Werte der letzen States aufsummiert eingetragen, also die Rewards der ganzen Weges